What is Image Augmentation 

Image Augmentation is the process of taking images that are already in a training dataset and manipulating them to create many altered versions of the same image. This both provides more images to train on, and can also help expose our classifier to a wider variety of lighting and coloring situations so as to make our classifier more robust.

What is the need of Image Augmentation ?

To combat the high expense of collecting thousands of training images, image augmentation has been developed in order to generate training data from an existing dataset.

Importing the libraries and data

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 25
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Splitting the data into training and test samples

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Convert class vectors to binary class metrices

In [4]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


Defining the CNN Architecture

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [6]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


Image Preprocessing and Data Augmentation

Running the Default Base Model

Creating an Image using ImageDataGenerator() 

To Augment our image data with keras, we simply call the ImageDataGenerator() function and pass a list of parameters describing the alterations that we want it to perform on the images. We will then call the fit() function on our image generator which will apply the changes to the images batch by batch. By default, the modifications will be applied randomly, so not every image will be changed every time. 

In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [8]:
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 2000, epochs= 25)

Epoch 1/25
2000/2000 [==============================] - 483s - loss: 1.8061 - acc: 0.3372   
Epoch 2/25
2000/2000 [==============================] - 480s - loss: 1.5327 - acc: 0.4412   
Epoch 3/25
2000/2000 [==============================] - 540s - loss: 1.3823 - acc: 0.5033   
Epoch 4/25
2000/2000 [==============================] - 533s - loss: 1.2947 - acc: 0.5363   
Epoch 5/25
2000/2000 [==============================] - 542s - loss: 1.2218 - acc: 0.5647   
Epoch 6/25
2000/2000 [==============================] - 509s - loss: 1.1560 - acc: 0.5921   
Epoch 7/25
2000/2000 [==============================] - 459s - loss: 1.1109 - acc: 0.6069   
Epoch 8/25
2000/2000 [==============================] - 506s - loss: 1.0657 - acc: 0.6230   
Epoch 9/25
2000/2000 [==============================] - 495s - loss: 1.0253 - acc: 0.6389   
Epoch 10/25
2000/2000 [==============================] - 534s - loss: 0.9910 - acc: 0.6519   
Epoch 11/25
2000/2000 [==============================] - 473s - loss:

Evaluating the model on test data

In [10]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])


Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 0.691914773178


Using the base mdoel, we were able to achieve an accuracy of 72% on training data and 69% on test data. Let us see the impact of changing the Image Generator parameters on accuracy in the cases below.

Case 1 : Rotating the images by 60 Degrees and Hz and vertical flip

In [11]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=60,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [12]:
# Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 2000, epochs= 25)

Epoch 1/25
2000/2000 [==============================] - 483s - loss: 1.5591 - acc: 0.4439   
Epoch 2/25
2000/2000 [==============================] - 498s - loss: 1.4716 - acc: 0.4749   
Epoch 3/25
2000/2000 [==============================] - 467s - loss: 1.4424 - acc: 0.4891   
Epoch 4/25
2000/2000 [==============================] - 475s - loss: 1.4124 - acc: 0.5000   
Epoch 5/25
2000/2000 [==============================] - 468s - loss: 1.3967 - acc: 0.5055   
Epoch 6/25
2000/2000 [==============================] - 473s - loss: 1.3832 - acc: 0.5105   
Epoch 7/25
2000/2000 [==============================] - 480s - loss: 1.3742 - acc: 0.5162   
Epoch 8/25
2000/2000 [==============================] - 513s - loss: 1.3598 - acc: 0.5204   
Epoch 9/25
2000/2000 [==============================] - 483s - loss: 1.3476 - acc: 0.5246   
Epoch 10/25
2000/2000 [==============================] - 512s - loss: 1.3473 - acc: 0.5267   
Epoch 11/25
2000/2000 [==============================] - 507s - loss:

In [13]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 1.04107242508
Test accuracy: 0.6404


The model is underfitting as test data accuracy is high as compared to training data, so training more samples would help.

Case 2 : Setting featurewise std normalization = true

In [14]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None)
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

In [15]:
 # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch= 2000, epochs= 25)

Epoch 1/25
2000/2000 [==============================] - 536s - loss: 1.1857 - acc: 0.6023   
Epoch 2/25
2000/2000 [==============================] - 533s - loss: 1.0774 - acc: 0.6336   
Epoch 3/25
2000/2000 [==============================] - 533s - loss: 1.0403 - acc: 0.6515   
Epoch 4/25
2000/2000 [==============================] - 537s - loss: 1.0175 - acc: 0.6581   
Epoch 5/25
2000/2000 [==============================] - 540s - loss: 1.0078 - acc: 0.6629   
Epoch 6/25
2000/2000 [==============================] - 529s - loss: 0.9932 - acc: 0.6670   
Epoch 7/25
2000/2000 [==============================] - 604s - loss: 0.9799 - acc: 0.6727   
Epoch 8/25
2000/2000 [==============================] - 652s - loss: 0.9699 - acc: 0.6743   
Epoch 9/25
2000/2000 [==============================] - 494s - loss: 0.9695 - acc: 0.6769   
Epoch 10/25
2000/2000 [==============================] - 644s - loss: 0.9638 - acc: 0.6789   
Epoch 11/25
2000/2000 [==============================] - 578s - loss:

In [16]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/neelambabel/Desktop/Cognitive Computing/saved_models/keras_cifar10_trained_model.h5 
 9984/10000 [============================>.] - ETA: 0sTest loss: 1.59539349689
Test accuracy: 0.4594


The model seems to be overfitting, hence it is better to train the model on more parameters to reduce the gap in train and test accuracy.